# パラメータ実験用のディレクトリを準備する。

本セクションでは、データセットの構造として、パラメータ実験用のデータ構成(RCOS_for_parameters)を指定した研究者向けに、  
同一の実験パッケージ内に、パラメータ実験用のディレクトリの準備を行います。

![パラメータ実験用実験リポジトリのイメージ](https://raw.githubusercontent.com/NII-DG/workflow-template/develop/fb44fb71/images/%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E5%AE%9F%E9%A8%93%E7%94%A8%E5%AE%9F%E9%A8%93%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E3%81%AE%E3%82%A4%E3%83%A1%E3%83%BC%E3%82%B8.png)



### - 0. セクションの実行対象確認

パラメータ実験用のデータ構成を利用していない方にとっては不要なセクションとなります。  
下記のセルを実行し、本セクションの実行対象かご確認ください。

In [ ]:
import os

os.chdir(path_flow_root)
from util.scripts import utils

# DS構成のスキーム名をパラメタファイルから取得する
monitoring_params = {}
with open(utils.fetch_monitoring_param_file_path(), mode='r') as f:
    monitoring_params = json.load(f)
    
scheme_name = monitoring_params['datasetStructure']

if scheme_name != 'RCOS_for_parameters':
    print( '本処理は不要ですので、実験中ワークフロー機能トップページに遷移してください。')

上記の処理で、「本処理は不要ですので、実験中ワークフロー機能トップページに遷移してください。」と表示された方は、  
[こちら](../experiment.ipynb)からトップページに遷移できます。

### - 1.1 パラメータ実験用の実験パッケージを用意する

In [ ]:
# このコンテナで扱う実験パッケージのパスを作成する
%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE

# /home/jovyan 配下に移動する 
%cd ~/

# 実験パッケージを用意する
!cp -r ~/WORKFLOWS/PACKAGE/scheme/$scheme_name/. $experiment_path
!cp -r ~/WORKFLOWS/PACKAGE/base/. $experiment_path


### 1.2 パラメータ実験名を決定する。

パラメータ実験名を入力して、実験用データ格納用のディレクトリを作成します。

パラメータ実験名としては、同じ実験パッケージ内で同盟のパラメータ実験名を使用することはできません。  
また、「parameter」というパラメータ実験名も使用することができません。パラメータ名など分かりやすい名前をご記入ください。  
※入力値に誤りがある場合、もう一度実行することで訂正ができます。  

![パラメータ実験用実験リポジトリのイメージ](https://raw.githubusercontent.com/NII-DG/workflow-template/develop/fb44fb71/images/%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E5%AE%9F%E9%A8%93%E7%94%A8%E5%AE%9F%E9%A8%93%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E3%81%AE%E3%82%A4%E3%83%A1%E3%83%BC%E3%82%B8.png)

In [ ]:
import os
import re
import shutil
from IPython.display import clear_output

# データセットの構成がパラメータ実験用ではない場合、何もしない。
if scheme_name != 'RCOS_for_parameters':
    print( '本処理は不要です。')
else:
    # GINサーバのものに合わせたバリデーションルールを設定
    validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

    print('作成したいパラメータ実験名を半角英数字で入力してください。')
    while True:
        paramexp_title = input("パラメータ実験名：")
        if validation.fullmatch(paramexp_title):
            break
        else:
            clear_output()
            print('パラメータ実験名は英数字、および"-", "_", "."のみで入力してください。')

    clear_output()

    # 実験中のワークフローに該当実験パッケージを特定させるため、環境変数EXPERIMENT_TITLEに実験パッケージ名を設定
    PARAMEXP_TITLE = paramexp_title
    %store PARAMEXP_TITLE
    clear_output()

    print("作成したパラメータ実験名：", paramexp_title)
    print('このパラメータ実験名で処理を進めます。')

    # 実験パッケージの直下に移動
    %cd $experiment_path

    # parameterディレクトリをユーザが指定したパラメータ実験名に変更
    shutil.move('parameter', paramexp_title)

## 2. 実行結果をデータガバナンス機能に同期する

ここまでの内容を保存し、データガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

%cd ~/
save_path = [ experiment_path + '/input_data/' + input_repo_title, '/home/jovyan/.gitmodules']
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_入力データの準備', PATH = save_path, IS_RECURSIVE = False)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

## 3. 実験中ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。  